<a href="https://colab.research.google.com/github/zhannatoleubek-png/special-okx-chainsaw/blob/main/%D0%94%D0%B8%D0%BF%D1%81%D0%B8%D0%BA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
import warnings
import time
import threading
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
warnings.filterwarnings('ignore')

class OKXFuturesClient:
    def __init__(self):
        self.base_url = "https://www.okx.com"

    def get_all_futures_symbols(self):
        """Получение списка всех фьючерсных пар с OKX"""
        try:
            url = f"{self.base_url}/api/v5/public/instruments"
            params = {'instType': 'SWAP'}  # Бессрочные фьючерсы

            response = requests.get(url, params=params, timeout=10)
            data = response.json()

            if data['code'] == '0':
                symbols = []
                for instrument in data['data']:
                    # Берем только USDT и USD пары с достаточной ликвидностью
                    if (instrument['instId'].endswith('-SWAP') and
                        (instrument['instId'].startswith('BTC') or
                         instrument['instId'].startswith('ETH') or
                         instrument['instId'].startswith('ADA') or
                         instrument['instId'].startswith('DOT') or
                         instrument['instId'].startswith('LINK') or
                         instrument['instId'].startswith('LTC') or
                         instrument['instId'].startswith('BCH') or
                         instrument['instId'].startswith('XRP') or
                         instrument['instId'].startswith('EOS') or
                         instrument['instId'].startswith('ETC') or
                         instrument['instId'].startswith('FIL') or
                         instrument['instId'].startswith('ATOM') or
                         instrument['instId'].startswith('SOL') or
                         instrument['instId'].startswith('DOGE') or
                         instrument['instId'].startswith('MATIC') or
                         instrument['instId'].startswith('AVAX') or
                         instrument['instId'].startswith('APT') or
                         instrument['instId'].startswith('ARB') or
                         instrument['instId'].startswith('OP'))):
                        symbols.append(instrument['instId'])

                print(f"✅ Найдено {len(symbols)} фьючерсных пар")
                return symbols[:30]  # Ограничиваем 30 парами для производительности
            else:
                print(f"❌ Ошибка получения списка пар: {data['msg']}")
                return self._get_default_symbols()

        except Exception as e:
            print(f"❌ Ошибка получения списка фьючерсных пар: {e}")
            return self._get_default_symbols()

    def _get_default_symbols(self):
        """Список пар по умолчанию"""
        return [
            "BTC-USD-SWAP", "ETH-USD-SWAP", "ADA-USD-SWAP", "DOT-USD-SWAP",
            "LINK-USD-SWAP", "LTC-USD-SWAP", "BCH-USD-SWAP", "XRP-USD-SWAP",
            "EOS-USD-SWAP", "ETC-USD-SWAP", "FIL-USD-SWAP", "ATOM-USD-SWAP",
            "SOL-USD-SWAP", "DOGE-USD-SWAP", "MATIC-USD-SWAP", "AVAX-USD-SWAP"
        ]

    def get_futures_candles(self, symbol='BTC-USD-SWAP', timeframe='5m', limit=100):
        """Получение фьючерсных данных для одной пары"""
        try:
            tf_mapping = {
                '1m': '1m', '3m': '3m', '5m': '5m', '15m': '15m',
                '30m': '30m', '1h': '1H', '4h': '4H', '1d': '1D'
            }

            okx_tf = tf_mapping.get(timeframe, '5m')

            url = f"{self.base_url}/api/v5/market/candles"
            params = {
                'instId': symbol,
                'bar': okx_tf,
                'limit': limit
            }

            response = requests.get(url, params=params, timeout=10)
            data = response.json()

            if data['code'] == '0' and data['data']:
                candles = data['data']
                candles.reverse()

                df_data = []
                for candle in candles:
                    df_data.append({
                        'Timestamp': datetime.fromtimestamp(int(candle[0]) / 1000),
                        'Open': float(candle[1]),
                        'High': float(candle[2]),
                        'Low': float(candle[3]),
                        'Close': float(candle[4]),
                        'Volume': float(candle[5]),
                        'VolumeCcy': float(candle[6]),
                        'OpenInterest': float(candle[7]) if len(candle) > 7 else 0
                    })

                df = pd.DataFrame(df_data)
                df.set_index('Timestamp', inplace=True)
                return df
            else:
                return None

        except Exception as e:
            print(f"❌ Ошибка получения данных для {symbol}: {e}")
            return None

    def get_funding_rate(self, symbol='BTC-USD-SWAP'):
        """Получение фандинг рейта"""
        try:
            url = f"{self.base_url}/api/v5/public/funding-rate"
            params = {'instId': symbol}

            response = requests.get(url, params=params, timeout=5)
            data = response.json()

            if data['code'] == '0' and data['data']:
                return float(data['data'][0]['fundingRate']) * 100
            else:
                return 0.01
        except:
            return 0.01

class TelegramBot:
    def __init__(self, bot_token, chat_id):
        self.bot_token = bot_token
        self.chat_id = chat_id
        self.base_url = f"https://api.telegram.org/bot{bot_token}/"

    def send_message(self, text):
        """Отправка сообщения в Telegram"""
        url = self.base_url + "sendMessage"
        payload = {
            'chat_id': self.chat_id,
            'text': text,
            'parse_mode': 'HTML'
        }
        try:
            response = requests.post(url, data=payload, timeout=10)
            return response.status_code == 200
        except Exception as e:
            print(f"❌ Ошибка отправки в Telegram: {e}")
            return False

    def send_futures_signal(self, signal_data):
        """Отправка фьючерсного сигнала"""
        emoji = "🟢" if signal_data['position_type'] == "LONG" else "🔴"

        # Создаем компактное сообщение для множественных сигналов
        message = f"""
{emoji} <b>ФЬЮЧЕРСНЫЙ СИГНАЛ</b> {emoji}

<b>🎯 {signal_data['futures_symbol']}</b>
<b>Позиция:</b> {signal_data['position_type']} | <b>Сила:</b> {signal_data['strength']}/5
<b>Цена:</b> ${signal_data['entry_price']:.2f} | <b>Плечо:</b> {signal_data['leverage']}x

<b>📊 Уровни:</b>
SL: ${signal_data['stop_loss']:.2f} | TP: ${signal_data['take_profit']:.2f}
R/R: {signal_data['rr_ratio']} | TF: {signal_data['timeframe']}

<b>📈 Индикаторы:</b> {signal_data['indicators']}
<b>🏦 Фандинг:</b> {signal_data['funding_rate']:+.4f}%

⏰ <b>Время:</b> {datetime.now().strftime('%H:%M:%S')}
        """

        return self.send_message(message)

    def send_market_summary(self, summary_data):
        """Отправка сводки по рынку"""
        message = f"""
<b>📊 СВОДКА ФЬЮЧЕРСНОГО РЫНКА</b>

<b>🔍 Проанализировано пар:</b> {summary_data['total_symbols']}
<b>🎯 Найдено сигналов:</b> {summary_data['total_signals']}
<b>🟢 LONG сигналов:</b> {summary_data['long_signals']}
<b>🔴 SHORT сигналов:</b> {summary_data['short_signals']}

<b>📈 Топ сигналов по силе:</b>
"""

        for signal in summary_data['top_signals']:
            emoji = "🟢" if signal['position_type'] == "LONG" else "🔴"
            message += f"{emoji} {signal['symbol']}: {signal['position_type']} (сила: {signal['strength']}/5)\n"

        message += f"\n⏰ <b>Обновлено:</b> {datetime.now().strftime('%H:%M:%S')}"

        return self.send_message(message)

class MultiFuturesAnalyzer:
    def __init__(self, telegram_bot=None):
        self.telegram_bot = telegram_bot
        self.okx_client = OKXFuturesClient()
        self.last_signals = {}
        self.analysis_count = 0

        # Параметры для скальпинга
        self.parameters = {
            'ema_fast': 5,
            'ema_slow': 13,
            'rsi_period': 11,
            'volume_ma': 8,
            'atr_period': 7
        }

    def calculate_indicators(self, df):
        """Расчет индикаторов для одной пары"""
        try:
            p = self.parameters

            # Базовые индикаторы
            df['ema_fast'] = df['Close'].ewm(span=p['ema_fast']).mean()
            df['ema_slow'] = df['Close'].ewm(span=p['ema_slow']).mean()

            # RSI
            delta = df['Close'].diff()
            gain = (delta.where(delta > 0, 0)).ewm(span=p['rsi_period']).mean()
            loss = (-delta.where(delta < 0, 0)).ewm(span=p['rsi_period']).mean()
            rs = gain / loss
            df['rsi'] = 100 - (100 / (1 + rs))

            # ATR
            high_low = df['High'] - df['Low']
            high_close = np.abs(df['High'] - df['Close'].shift())
            low_close = np.abs(df['Low'] - df['Close'].shift())
            true_range = np.maximum(np.maximum(high_low, high_close), low_close)
            df['atr'] = true_range.ewm(span=p['atr_period']).mean()

            # Volume
            df['volume_ma'] = df['Volume'].ewm(span=p['volume_ma']).mean()
            df['volume_ratio'] = df['Volume'] / df['volume_ma']

            # MACD
            df['macd_fast'] = df['Close'].ewm(span=6).mean()
            df['macd_slow'] = df['Close'].ewm(span=13).mean()
            df['macd'] = df['macd_fast'] - df['macd_slow']
            df['macd_signal'] = df['macd'].ewm(span=5).mean()

            return df
        except Exception as e:
            print(f"❌ Ошибка расчета индикаторов: {e}")
            return None

    def analyze_symbol(self, symbol, timeframe="5m", leverage=5):
        """Анализ одной фьючерсной пары"""
        try:
            # Получаем данные
            data = self.okx_client.get_futures_candles(symbol, timeframe, 100)
            if data is None or data.empty:
                return None

            # Расчет индикаторов
            data = self.calculate_indicators(data)
            if data is None:
                return None

            data = data.dropna()
            if len(data) < 20:
                return None

            # Получаем текущие значения
            current = data.iloc[-1]
            prev = data.iloc[-2]

            # Генерация сигналов
            signals = self._generate_signals_for_symbol(symbol, current, prev, timeframe, leverage)

            return signals

        except Exception as e:
            print(f"❌ Ошибка анализа {symbol}: {e}")
            return None

    def _generate_signals_for_symbol(self, symbol, current, prev, timeframe, leverage):
        """Генерация сигналов для одной пары"""
        long_conditions = 0
        short_conditions = 0
        indicators = []

        # Условие 1: EMA пересечение
        if current['ema_fast'] > current['ema_slow'] and prev['ema_fast'] <= prev['ema_slow']:
            long_conditions += 1
            indicators.append("EMA")
        elif current['ema_fast'] < current['ema_slow'] and prev['ema_fast'] >= prev['ema_slow']:
            short_conditions += 1
            indicators.append("EMA")

        # Условие 2: RSI
        if current['rsi'] < 32 and prev['rsi'] >= 32:
            long_conditions += 1
            indicators.append("RSI")
        elif current['rsi'] > 68 and prev['rsi'] <= 68:
            short_conditions += 1
            indicators.append("RSI")

        # Условие 3: MACD
        if current['macd'] > current['macd_signal'] and prev['macd'] <= prev['macd_signal']:
            long_conditions += 1
            indicators.append("MACD")
        elif current['macd'] < current['macd_signal'] and prev['macd'] >= prev['macd_signal']:
            short_conditions += 1
            indicators.append("MACD")

        # Условие 4: Volume
        if current['volume_ratio'] > 2.0:
            if long_conditions > short_conditions:
                long_conditions += 1
                indicators.append("VOL")
            elif short_conditions > long_conditions:
                short_conditions += 1
                indicators.append("VOL")

        signals = []

        # Генерация LONG сигнала
        if long_conditions >= 2 and len(indicators) >= 2:
            leverage_factor = min(leverage / 10, 1.5)
            stop_loss_atr = 1.5 / leverage_factor

            signal_data = {
                'timestamp': datetime.now(),
                'position_type': 'LONG',
                'entry_price': float(current['Close']),
                'strength': min(5, int(long_conditions)),
                'stop_loss': float(current['Close'] - stop_loss_atr * current['atr']),
                'take_profit': float(current['Close'] + 2.5 * current['atr']),
                'indicators': ', '.join(set(indicators)),
                'timeframe': timeframe,
                'leverage': leverage,
                'rr_ratio': "1:1.7",
                'futures_symbol': symbol,
                'funding_rate': self.okx_client.get_funding_rate(symbol)
            }
            signals.append(signal_data)

        # Генерация SHORT сигнала
        elif short_conditions >= 2 and len(indicators) >= 2:
            leverage_factor = min(leverage / 10, 1.5)
            stop_loss_atr = 1.5 / leverage_factor

            signal_data = {
                'timestamp': datetime.now(),
                'position_type': 'SHORT',
                'entry_price': float(current['Close']),
                'strength': min(5, int(short_conditions)),
                'stop_loss': float(current['Close'] + stop_loss_atr * current['atr']),
                'take_profit': float(current['Close'] - 2.5 * current['atr']),
                'indicators': ', '.join(set(indicators)),
                'timeframe': timeframe,
                'leverage': leverage,
                'rr_ratio': "1:1.7",
                'futures_symbol': symbol,
                'funding_rate': self.okx_client.get_funding_rate(symbol)
            }
            signals.append(signal_data)

        return signals

    def analyze_all_symbols(self, timeframe="5m", leverage=5, max_workers=10):
        """Анализ всех фьючерсных пар с использованием многопоточности"""
        print(f"\n=== АНАЛИЗ ВСЕХ ФЬЮЧЕРСНЫХ ПАР ===")

        # Получаем список всех пар
        symbols = self.okx_client.get_all_futures_symbols()
        print(f"🔍 Анализ {len(symbols)} фьючерсных пар...")

        all_signals = []
        analyzed_count = 0

        # Используем многопоточность для ускорения анализа
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_symbol = {
                executor.submit(self.analyze_symbol, symbol, timeframe, leverage): symbol
                for symbol in symbols
            }

            for future in as_completed(future_to_symbol):
                symbol = future_to_symbol[future]
                try:
                    signals = future.result()
                    if signals:
                        all_signals.extend(signals)
                        print(f"✅ {symbol}: {len(signals)} сигналов")
                    else:
                        print(f"➖ {symbol}: сигналов нет")
                    analyzed_count += 1
                except Exception as e:
                    print(f"❌ Ошибка анализа {symbol}: {e}")
                    analyzed_count += 1

        print(f"\n📊 Анализ завершен: {analyzed_count}/{len(symbols)} пар")
        print(f"🎯 Всего сигналов: {len(all_signals)}")

        # Сортируем сигналы по силе (от самых сильных)
        all_signals.sort(key=lambda x: x['strength'], reverse=True)

        return all_signals

    def send_signals_to_telegram(self, signals, max_signals_per_batch=5):
        """Отправка сигналов в Telegram с ограничением количества"""
        if not signals or not self.telegram_bot:
            return 0

        sent_count = 0
        # Берем только самые сильные сигналы
        top_signals = signals[:max_signals_per_batch]

        for signal in top_signals:
            # Проверяем, не отправляли ли мы уже этот сигнал
            signal_key = f"{signal['futures_symbol']}_{signal['position_type']}"
            last_signal_time = self.last_signals.get(signal_key)

            if last_signal_time and (datetime.now() - last_signal_time).total_seconds() < 1800:  # 30 минут
                continue

            if self.telegram_bot.send_futures_signal(signal):
                sent_count += 1
                self.last_signals[signal_key] = datetime.now()
                print(f"📤 Отправлен сигнал для {signal['futures_symbol']}")
                time.sleep(1)  # Пауза между отправками

        return sent_count

    def send_market_summary_to_telegram(self, signals, total_symbols):
        """Отправка сводки по рынку"""
        if not self.telegram_bot:
            return

        long_signals = [s for s in signals if s['position_type'] == 'LONG']
        short_signals = [s for s in signals if s['position_type'] == 'SHORT']

        summary_data = {
            'total_symbols': total_symbols,
            'total_signals': len(signals),
            'long_signals': len(long_signals),
            'short_signals': len(short_signals),
            'top_signals': [
                {
                    'symbol': s['futures_symbol'],
                    'position_type': s['position_type'],
                    'strength': s['strength']
                }
                for s in signals[:5]  # Топ-5 сигналов
            ]
        }

        self.telegram_bot.send_market_summary(summary_data)

def test_telegram_connection(bot_token, chat_id):
    """Тестирование подключения к Telegram"""
    print("🔍 Тестирование подключения к Telegram...")
    bot = TelegramBot(bot_token, chat_id)
    test_message = "🤖 <b>Тестовое сообщение от Multi-Futures Bot</b>\n\n✅ Бот запущен и готов к анализу всех фьючерсных пар!"
    result = bot.send_message(test_message)

    if result:
        print("✅ Подключение к Telegram успешно!")
        return True
    else:
        print("❌ Ошибка подключения к Telegram!")
        return False

# ОСНОВНОЙ ЗАПУСК МУЛЬТИФЬЮЧЕРСНОГО АНАЛИЗАТОРА
if __name__ == "__main__":
    # === НАСТРОЙКИ ===
    BOT_TOKEN = "8294892098:AAFX0Zzq9yN1on6UlID8f7vzif4dWR_7uWs"  # Замените на реальный токен
    CHAT_ID = "381202205"      # Замените на реальный chat_id

    TIMEFRAME = "5m"
    LEVERAGE = 5  # Плечо по умолчанию
    ANALYSIS_INTERVAL = 300  # Интервал анализа в секундах (5 минут)
    MAX_SIGNALS_PER_BATCH = 8  # Максимум сигналов за один цикл

    print("🚀 ЗАПУСК MULTI-FUTURES SCALPING BOT")
    print("=" * 50)

    # Инициализация бота
    telegram_bot = TelegramBot(BOT_TOKEN, CHAT_ID)

    # Тестирование подключения
    if test_telegram_connection(BOT_TOKEN, CHAT_ID):
        print("\n✅ Все системы готовы к работе!")

        # Инициализация анализатора
        analyzer = MultiFuturesAnalyzer(telegram_bot)

        # Счетчик циклов
        cycle_count = 0

        while True:
            try:
                cycle_count += 1
                print(f"\n🔄 ЦИКЛ АНАЛИЗА #{cycle_count}")
                print("=" * 30)

                # Анализ всех фьючерсных пар
                start_time = time.time()
                all_signals = analyzer.analyze_all_symbols(TIMEFRAME, LEVERAGE, max_workers=15)
                analysis_time = time.time() - start_time

                print(f"⏱️ Время анализа: {analysis_time:.2f} сек")

                # Отправка сигналов в Telegram
                if all_signals:
                    sent_count = analyzer.send_signals_to_telegram(all_signals, MAX_SIGNALS_PER_BATCH)
                    print(f"📤 Отправлено сигналов: {sent_count}/{len(all_signals)}")

                    # Отправляем сводку каждые 3 цикла или если есть сигналы
                    if cycle_count % 3 == 0 or sent_count > 0:
                        analyzer.send_market_summary_to_telegram(all_signals, len(analyzer.okx_client.get_all_futures_symbols()))
                else:
                    print("➖ Сигналов не найдено")
                    # Отправляем сводку каждые 5 циклов даже если нет сигналов
                    if cycle_count % 5 == 0:
                        analyzer.send_market_summary_to_telegram([], len(analyzer.okx_client.get_all_futures_symbols()))

                # Расчет времени до следующего анализа
                sleep_time = max(ANALYSIS_INTERVAL - analysis_time, 60)
                print(f"💤 Следующий анализ через {sleep_time:.0f} сек...")
                time.sleep(sleep_time)

            except KeyboardInterrupt:
                print("\n⏹️ Остановка мультифьючерсного бота...")
                break
            except Exception as e:
                print(f"❌ Ошибка в основном цикле: {e}")
                print("💤 Повторная попытка через 60 сек...")
                time.sleep(60)
    else:
        print("\n❌ Проверьте настройки Telegram бота!")

🚀 ЗАПУСК MULTI-FUTURES SCALPING BOT
🔍 Тестирование подключения к Telegram...
✅ Подключение к Telegram успешно!

✅ Все системы готовы к работе!

🔄 ЦИКЛ АНАЛИЗА #1

=== АНАЛИЗ ВСЕХ ФЬЮЧЕРСНЫХ ПАР ===
✅ Найдено 36 фьючерсных пар
🔍 Анализ 30 фьючерсных пар...
➖ XRP-USD-SWAP: сигналов нет
➖ ADA-USD-SWAP: сигналов нет
➖ AVAX-USD-SWAP: сигналов нет
➖ LTC-USD-SWAP: сигналов нет
➖ DOT-USD-SWAP: сигналов нет
➖ SOL-USD-SWAP: сигналов нет
➖ DOGE-USD-SWAP: сигналов нет
➖ BTC-USD-SWAP: сигналов нет
➖ BTC-USDT-SWAP: сигналов нет
➖ SOL-USD_UM-SWAP: сигналов нет
➖ LINK-USD-SWAP: сигналов нет
➖ FIL-USD-SWAP: сигналов нет
➖ ETC-USD-SWAP: сигналов нет
➖ BCH-USD-SWAP: сигналов нет
➖ ETH-USD-SWAP: сигналов нет
➖ SOL-USDT-SWAP: сигналов нет
➖ BCH-USDT-SWAP: сигналов нет
➖ ETH-USDT-SWAP: сигналов нет
➖ APT-USDT-SWAP: сигналов нет
➖ ETHW-USDT-SWAP: сигналов нет
➖ DOT-USDT-SWAP: сигналов нет
➖ ETC-USDT-SWAP: сигналов нет
➖ ARB-USDT-SWAP: сигналов нет
➖ DOGE-USDT-SWAP: сигналов нет
➖ FIL-USDT-SWAP: сигналов нет
